# Lab 12: Lists, Iteration, and Functional Programming

In [1]:
library(tidyverse)
library(nycflights13)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.5
✔ tidyr   0.8.1     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.3.0
Warning message:
“package ‘tibble’ was built under R version 3.4.3”Warning message:
“package ‘tidyr’ was built under R version 3.4.4”Warning message:
“package ‘purrr’ was built under R version 3.4.4”Warning message:
“package ‘dplyr’ was built under R version 3.4.4”Warning message:
“package ‘forcats’ was built under R version 3.4.3”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message:
“package ‘nycflights13’ was built under R version 3.4.4”

## Vectors and Lists

**Vectors** are sequences of objects, all of which have the same type. Recall that we can assign these with the `c()` operator:
```
x = c(1, 2, 3, 4)
x = 1:4
x = c('a', 'b', 'c')


```

We can also name the entries in a vector either while assigning or by using the `names()` function:
```
x = c(a=1, b=2, c=3)

x = c(1, 2, 3)
names(x) = c('a', 'b', 'c')
```


**Lists** are also sequences, but the elements of a list are allowed to be different data types (including vectors or even other lists). Like with vectors, list elements can be named.

In [52]:
x = list(a='a', b=FALSE, c=pi, d=1:3)
print(x)
print(names(x))

$a
[1] "a"

$b
[1] FALSE

$c
[1] 3.141593

$d
[1] 1 2 3

[1] "a" "b" "c" "d"


There are three ways to subset or extract elements from a list:
* `[...]` will extract a sublist. Note that the result of this will always be another list. Integer, logical, or character vectors can be used.
* `[[...]]` will extract a single element. Either the index or name of the desired element can be provided.
* `$a` will also extract a single element. Note that this requires a named list, and the name must be used. 

In [44]:
print(x[1:2])

$a
[1] "a"

$b
[1] FALSE



In [49]:
print(x['a'])

$a
[1] "a"



In [48]:
print(x[['a']])

[1] "a"


In [51]:
print(x$a)

[1] "a"


Note that dataframes (and tibbles) are basically just lists with some extra features. Here the `names()` correspond to columns, where each element is a vector.

## Iteration

The two main ways of iterating in R are `for` and `while` loops.

```
for (index in vector) {
    [do something for each index]
}
```

```
while (condition) {
    [do something]
}
```

Note that when using a `while` loop it will keep running as long as the condition is `TRUE`, so you want to make sure that eventually whatever you do in the loop will make the condition become `FALSE`.


## Functional Programming

Tidyverse contains a suite of functions used for functional programming in `purrr` ([documentation](https://purrr.tidyverse.org/index.html)).

Functional programming is generally built on three main operations:
* `map`
* `keep` (usually known as `filter` in other languages)
* `reduce`

Note that `map` always returns a list, if you want a vector, then use the functions `map_lgl`, `map_int`, `map_dbl`, or `map_chr` for logicals, integers, doubles/floats, and strings, respectively.

In [183]:
map(1:5, function(x) x^2) %>% print

[[1]]
[1] 1

[[2]]
[1] 4

[[3]]
[1] 9

[[4]]
[1] 16

[[5]]
[1] 25



In [188]:
map_dbl(1:5, function(x) x^2)

[1]  1  4  9 16 25

In [193]:
keep(1:5, function(x) x %% 2 == 0)

[1] 2 4

In [186]:
reduce(1:5, function(x, y) x + y)

[1] 15

In [187]:
accumulate(1:5, function(x, y) x + y)

[1]  1  3  6 10 15

In [196]:
map_dbl(1:5, ~ .^2)
keep(1:5, ~ . %% 2 == 0)
reduce(1:5, ~ .x + .y)

[1]  1  4  9 16 25

[1] 2 4

[1] 15

Remember that dataframes are also lists where each element is a vector (i.e. a column of the data), so when applied to dataframes these will apply to each column of the data.

In [198]:
mtcars %>% map_dbl(sum) %>% keep(~ . > 200) %>% print

    mpg    disp      hp    qsec 
 642.90 7383.10 4694.00  571.16 


## Memoization

Memoization is a technique for speeding up computation (especially when used with recursion) where we store the result to arguments we've already seen. Consider a recursive solution to calculating the Fibonacci sequence:

In [161]:
fib = function(n) {
    if (n == 1) return(1)
    if (n == 2) return(1)
    
    return(fib(n-1) + fib(n-2))
}
fib(10)

[1] 55

Note all the repeated calculation that happens when we call this function. Is there a way to not do all this repetition?

In [165]:
memostor_fib = rep(NA, 1000) # Only works for n <= 1000
fibmemo = function(n) {
    if (n == 1) return(1)
    if (n == 2) return(1)
    if (is.na(memostor_fib[n])) {
        memostor_fib[n] <<- fibmemo(n-1) + fibmemo(n-2)
    }
    return(memostor_fib[n])
}
fibmemo(10)

[1] 55

In [160]:
library(microbenchmark)
microbenchmark(
    fib(20),
    fibmemo(20), unit='us'
) %>% print

Unit: microseconds
        expr      min       lq       mean    median       uq      max neval cld
     fib(20) 5001.906 5150.373 5483.25065 5402.2395 5590.400 7020.209   100   b
 fibmemo(20)    0.531    0.809    3.35476    1.5885    6.372    9.740   100  a 


## Q1: Given a vector `x`, write a function that calculates the sum of the squared odd entries.

For example:
```
f(c(1, 2, 3, 4, 5) == 1^2 + 3^2 + 5^2 == 35
f(c(2, 4, 6)) == 0
```

Do this using loops and if statements in `f_loop`, and using functional programming tools (`map`, `reduce`, `keep`) in `f_func`.

In [123]:
f_loop = function(x) {

}

In [199]:
f_func = function(x) {

}

In [122]:
stopifnot(f_loop(c(1, 2, 3, 4, 5)) == 35)
stopifnot(f_func(c(1, 2, 3, 4, 5)) == 35)

stopifnot(f_loop(c(2, 4, 6)) == 0)
stopifnot(f_func(c(2, 4, 6)) == 0)

## Q2: Use a loop to do the following, storing the result in a vector.

1. Compute the mean of every column in `airquality` ignoring missing values.
2. Compute the number of unique values in each column of `iris`.

In [1]:
# Q2.1


In [2]:
# Q2.2


## Q3: Repeat Question 2 using functional programming instead of loops

In [4]:
#Q3.1


In [3]:
#Q3.2


## Q4: Recursion and memoization

Suppose Roger can climb either 1 or 2 steps at a time. Consider a function that returns how many ways Roger fan climb up `n` steps. For example, if `n=3`, then there are 3 ways: `(1, 1, 1)`, `(1, 2)`, or `(2, 1)`.

Write a function that performs this calculation using recursion:

Note that if `n` is large there will be a lot of repeated calculation. Can you use memoization to speed this up?